In [ ]:
import warnings
warnings.filterwarnings('ignore', 'numpy.dtype size changed')
warnings.filterwarnings( 'ignore', category=FutureWarning)

from datetime import datetime, timedelta

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.util as cutil
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import metpy.calc as mpcalc
from metpy.interpolate import interpolate_to_isosurface
import metpy.constants as mpconstants
from metpy.units import units
import numpy as np
from pyproj import Proj
from scipy.ndimage import gaussian_filter
import xarray as xr

In [ ]:
def plot_PV(level, date):
    print("Creating the {}-hPa PV Map".format(level))
    ilev = list(lev.m).index(level*100.)

    uwnd_ilev = uwnd[ilev].metpy.convert_units('kt')
    vwnd_ilev = vwnd[ilev].metpy.convert_units('kt')
    
    sped_ilev = mpcalc.wind_speed(uwnd_ilev, vwnd_ilev)
    div_ilev = mpcalc.smooth_n_point(div[ilev], 9, 2)
    
    epv_smooth = mpcalc.smooth_n_point(epv[ilev], 9, 2)
    
    diff_epv_smooth = mpcalc.smooth_n_point(diff_epv[ilev], 9, 2)
    
    fig = plt.figure(1, figsize=(26, 21))
    fig.subplots_adjust(hspace=0.02, wspace=0.05)
    
    # 1st panel
    ax = plt.subplot(221, projection=mapcrs)
    ax.set_extent([-130, -72, 20, 55], ccrs.PlateCarree())
    ax.add_feature(cfeature.COASTLINE.with_scale('50m'))
    ax.add_feature(cfeature.STATES.with_scale('50m'))

    cf = ax.contourf(clons, clats, sped_ilev, range(10,230,20), cmap=plt.cm.BuPu)
    plt.colorbar(cf, orientation='horizontal', pad=0, aspect=50, extendrect=True)

    cs = ax.contour(clons, clats, epv_smooth*1e6, range(2,15,1), colors='black')
    plt.clabel(cs, fmt='%d')
    
    cs2 = ax.contour(clons, clats, div_ilev*1e5, range(1,50,3), colors='grey', linestyles='dashed')
    plt.clabel(cs2, fmt='%d')

#     ax.barbs(lons[wind_slice], lats[wind_slice],
#              uwnd_ilev[wind_slice].m, vwnd_ilev[wind_slice].m,
#              transform=ccrs.PlateCarree())

    plt.title(f'{int(lev[ilev].m/100)}-hPa PV (PVU), Divergence ($*10^5$ $s^{-1}$), and Wind Spped (kt)', loc='left')
    plt.title(f'Valid Time: {vtime}', loc='right')
    
    
    # 2nd panel
    ax = plt.subplot(222, projection=mapcrs)
    ax.set_extent([-130, -72, 20, 55], ccrs.PlateCarree())
    ax.add_feature(cfeature.COASTLINE.with_scale('50m'))
    ax.add_feature(cfeature.STATES.with_scale('50m'))

    cs = ax.contour(clons, clats, epv_smooth*1e6, range(2,15,1), colors='black')
    plt.clabel(cs, fmt='%d')
    
    cf = ax.contourf(clons, clats, diff_epv_smooth*1e6, range(-15,16,1), cmap=plt.cm.RdBu_r, extend='both')
    plt.colorbar(cf, orientation='horizontal', pad=0, aspect=50, extendrect=True)

    plt.title(f'{int(lev[ilev].m/100)}-hPa 6 h PV Difference', loc='left')
    plt.title(f'Valid Time: {vtime}', loc='right')
    
    # 3rd panel
    ax = plt.subplot(223, projection=mapcrs)
    ax.set_extent([-130, -72, 20, 55], ccrs.PlateCarree())
    ax.add_feature(cfeature.COASTLINE.with_scale('50m'))
    ax.add_feature(cfeature.STATES.with_scale('50m'))

    cs = ax.contour(clons, clats, epv_smooth*1e6, range(2,15,1), colors='black')
    plt.clabel(cs, fmt='%d')
    
    cf = ax.contourf(clons, clats, avg_adv_epv[ilev], range(-15,16,1), cmap=plt.cm.RdBu_r, extend='both')
    plt.colorbar(cf, orientation='horizontal', pad=0, aspect=50, extendrect=True)

    plt.title(f'{int(lev[ilev].m/100)}-hPa Est. 6-h PV Advection', loc='left')
    plt.title(f'Valid Time: {vtime}', loc='right')
    
    
    # 4th panel
    ax = plt.subplot(224, projection=mapcrs)
    ax.set_extent([-130, -72, 20, 55], ccrs.PlateCarree())
    ax.add_feature(cfeature.COASTLINE.with_scale('50m'))
    ax.add_feature(cfeature.STATES.with_scale('50m'))

    cs = ax.contour(clons, clats, epv_smooth*1e6, range(2,15,1), colors='black')
    plt.clabel(cs, fmt='%d')
    
    cf = ax.contourf(clons, clats, dbh_epv[ilev], range(-15,16,1), cmap=plt.cm.RdBu_r, extend='both')
    plt.colorbar(cf, orientation='horizontal', pad=0, aspect=50, extendrect=True)

    plt.title(f'{int(lev[ilev].m/100)}-hPa Est. Diabatic Heating PV', loc='left')
    plt.title(f'Valid Time: {vtime}', loc='right')
    

    plt.savefig(f'{int(lev[ilev].m/100)}-hPa_four_panel_DBH_PV_{date:%Y%m%d_%H}00.png', bbox_inches='tight', dpi=150)
    plt.show()
    #plt.close()

In [ ]:
date = datetime(2010, 10, 25, 0)

while date <= datetime(2010, 10, 27, 0):
    prev_date = date - timedelta(hours=6)

    ds = xr.open_dataset('http://www.ncei.noaa.gov/thredds/dodsC/model-gfs-g4-anl-files-old/'
                         f'{date:%Y%m}/{date:%Y%m%d}/gfsanl_4_{date:%Y%m%d}_{date:%H}00_000.grb2').metpy.parse_cf()

    ds_prev = xr.open_dataset('http://www.ncei.noaa.gov/thredds/dodsC/model-gfs-g4-anl-files-old/'
                              f'{prev_date:%Y%m}/{prev_date:%Y%m%d}/gfsanl_4_{prev_date:%Y%m%d}_{prev_date:%H}00_000.grb2').metpy.parse_cf()

    # ds = xr.open_dataset('groundhogs_day_blizzard/GFS_{0:%Y%m%d}_{0:%H}00.nc'.format(date))
    lev_name = ds.Temperature_isobaric.dims[1]
    prev_lev_name = ds_prev.Temperature_isobaric.dims[1]

    ip100_3 = list(ds[lev_name].values).index(10000)-1
    prev_ip100_3 = list(ds_prev[prev_lev_name].values).index(10000)-1
    #ip100_5 = list(ds.isobaric5.values).index(10000)-1

    lev = ds[lev_name].values[ip100_3:] * units.Pa
    prev_lev = ds_prev[prev_lev_name].values[prev_ip100_3:] * units.Pa

    wind_slice = (slice(None, None, 5), slice(None, None, 5))

    lats = ds.lat
    lons = cutil.add_cyclic_point(ds.lon)

    # Set subset slice for the geographic extent of data to limit download
    lon_slice = slice(400,701)
    lat_slice = slice(10,160)

    lat1d = lats[lat_slice]
    lon1d = lons[lon_slice]
    lons, lats = np.meshgrid(lon1d, lat1d)

    dx, dy = mpcalc.lat_lon_grid_deltas(lons, lats)

    uwnd = ds['u-component_of_wind_isobaric'][0, ip100_3:, lat_slice, lon_slice].metpy.quantify()
    vwnd = ds['v-component_of_wind_isobaric'][0, ip100_3:, lat_slice, lon_slice].metpy.quantify()
    tmpk = ds['Temperature_isobaric'][0, ip100_3:, lat_slice, lon_slice].metpy.quantify()

    prev_uwnd = ds_prev['u-component_of_wind_isobaric'][0, prev_ip100_3:, lat_slice, lon_slice].metpy.quantify()
    prev_vwnd = ds_prev['v-component_of_wind_isobaric'][0, prev_ip100_3:, lat_slice, lon_slice].metpy.quantify()
    prev_tmpk = ds_prev['Temperature_isobaric'][0, prev_ip100_3:, lat_slice, lon_slice].metpy.quantify()

    vtime = ds.Geopotential_height_isobaric.time.data[0].astype('datetime64[ms]').astype('O')

    mapcrs = ccrs.LambertConformal(central_longitude=-100, central_latitude=35, standard_parallels=(30, 60))
    datacrs = ccrs.PlateCarree()

    # Transform Coordinates ahead of time
    tlatlons = mapcrs.transform_points(ccrs.PlateCarree(), lons, lats)
    clons = tlatlons[:,:,0]
    clats = tlatlons[:,:,1]

    thta = mpcalc.potential_temperature(lev[:, None, None], tmpk)

    relvor = mpcalc.vorticity(uwnd, vwnd)

    div = mpcalc.divergence(uwnd, vwnd)

    epv = mpcalc.potential_vorticity_baroclinic(thta, lev, uwnd, vwnd)

    ip850 = list(lev.m).index(850*100.)
    ip925 = list(lev.m).index(925*100.)
    relvor_925850 = np.average(relvor[[ip925,ip850],:,:], axis=0)


    prev_thta = mpcalc.potential_temperature(prev_lev[:, None, None], prev_tmpk)

    prev_relvor = mpcalc.vorticity(prev_uwnd, prev_vwnd)

    prev_div = mpcalc.divergence(prev_uwnd, prev_vwnd)

    prev_epv = mpcalc.potential_vorticity_baroclinic(prev_thta, prev_lev, prev_uwnd, prev_vwnd)

    prev_ip850 = list(prev_lev.m).index(850*100.)
    prev_ip925 = list(prev_lev.m).index(925*100.)
    prev_relvor_925850 = np.average(prev_relvor[[prev_ip925,prev_ip850],:,:], axis=0)

    diff_epv = epv - prev_epv

    prev_adv_epv = (mpcalc.advection(prev_epv*1e6, prev_uwnd, prev_vwnd, dx=dx[None, :, :], dy=dy[None, :, :]))
    #adv_epv = (mpcalc.advection(epv*1e6, uwnd, vwnd, dx=dx[None, :, :], dy=dy[None, :, :]))
    #avg_adv_epv = mpcalc.smooth_n_point(((prev_adv_epv + adv_epv) / 2 * (6*3600 * units.seconds)), 9, 10)
    avg_adv_epv =  mpcalc.smooth_n_point(prev_adv_epv, 9, 10) * (6*3600 * units.seconds)

    dbh_epv = diff_epv - avg_adv_epv

    plot_PV(250, date)
    
    date += timedelta(hours=6)